# Trend Data

## Single Metric Trends

In [2]:
from src.ingest_data.structured.files import get_csv_data
cpi = get_csv_data(".\src\data\CPIAUCSL.csv") # https://fred.stlouisfed.org/series/CPIAUCSL
cpi.tail(5)

,DATE,CPIAUCSL
916,2023-05-01,303.294
917,2023-06-01,303.841
918,2023-07-01,304.348
919,2023-08-01,306.269
920,2023-09-01,307.481


In [3]:
from src.trend_data.trend import get_trend_line, get_trend_bar
get_trend_line(cpi, x_axis="DATE", y_axis="CPIAUCSL", title="U.S. Federal Reserve's Monthly Consumer Price Index - Line Chart")

In [4]:
get_trend_bar(cpi, x_axis="DATE", y_axis="CPIAUCSL", title="U.S. Federal Reserve's Monthly Consumer Price Index - Bar Chart")

## Single Metric Trends by Dimension

In [6]:
from src.ingest_data.structured.files import get_json_data
history = get_json_data(method="custom",path=".\src\data\en_historic_events.json", orient="index")
history.sample(5)

,date,description,lang,category1,category2,granularity
6761,863,"After the death of Charles, Lothair takes part...",en,By place,Europe,year
3370,280,"Julius Saturninus, governor of Syria, is in Al...",en,By place,Roman Empire,year
1142,-65,Pompey the Great subjugates Kingdom of Iberia ...,en,By place,Roman Republic,year
14341,1616/12/22,"In Tunis, Tunisia, the mosque of Youssef Deyis...",en,Date unknown,NaN,year
32679,1978/03/17,California Jam II is held at the Ontario Motor...,en,March,NaN,year


In [7]:
from src.utils.df_format_utils import transform_series_to_year, group_trend
history_w_year = transform_series_to_year(data=history, date_column="date")
display(history_w_year)

,date,description,lang,category1,category2,granularity,year
0,-300,Pilgrims travel to the healing temples of Ascl...,en,By place,Greece,year,-300
1,-300,"Pyrrhus, the King of Epirus, is taken as a hos...",en,By place,Egypt,year,-300
2,-300,Ptolemy concludes an alliance with King Lysima...,en,By place,Egypt,year,-300
3,-300,"Seleucus founds the city of Antioch, some 20 m...",en,By place,Seleucid Empire,year,-300
4,-300,"After the death of his wife Apama, Seleucus ma...",en,By place,Seleucid Empire,year,-300
...,...,...,...,...,...,...,...
10465,1299/12/01,The city of Florence bans the use of Arabic nu...,en,By topic,Markets,year,1299
24086,1921,"Hyperinflation is still rampant in Germany, wh...",en,November,Undated,year,1921
24168,1922/08/28,Hyperinflation in Germany has seen the value o...,en,August,Undated,year,1922
24186,1922/10/30,"3,000 German marks are now needed to buy a sin...",en,October,Undated,year,1922


In [15]:
history_grouped = history_w_year.groupby(by=["year","category2"]).agg({"description":"count"})    
history_grouped.reset_index(inplace=True)
history_grouped['event_count'] = history_grouped['description']
history_grouped


,year,category2,description,event_count
0,-300,Art,1,1
1,-300,Egypt,2,2
2,-300,Greece,1,1
3,-300,India,1,1
4,-300,Seleucid Empire,2,2
...,...,...,...,...
4220,1299,Asia,2,2
4221,1299,Europe,8,8
4222,1299,Markets,1,1
4223,1921,Undated,1,1


In [27]:
get_trend_bar(history_grouped, x_axis="year", y_axis="event_count", color="category2", title="World History Events - Dimension Grouped Bar Chart")

In [30]:
history_grouped['category2'].value_counts()

category2
Asia                              732
Europe                            665
Religion                          628
Roman Empire                      459
Roman Republic                    214
                                 ... 
Persian and Byzantine Empires       1
Ancient Japan                       1
Visigothic kingdom                  1
Eastern Roman/Byzantine Empire      1
Culture and Religion                1
Name: count, Length: 172, dtype: int64

In [63]:
from src.trend_data.trend import get_trend_distribution
asia = history_grouped.loc[history_grouped["category2"]=="Asia"]
asia

,year,category2,description,event_count
253,-214,Asia,3,3
485,-140,Asia,1,1
492,-138,Asia,2,2
506,-129,Asia,1,1
523,-121,Asia,1,1
...,...,...,...,...
4205,1290,Asia,2,2
4207,1291,Asia,3,3
4212,1293,Asia,4,4
4215,1298,Asia,1,1


In [64]:
asia = history_grouped.loc[history_grouped['category2']=="Asia"]
get_trend_distribution(asia, "year")

In [67]:
top = history_grouped.loc[history_grouped["category2"].isin(["Asia","Europe","Religion","Roman Empire"])]
get_trend_distribution(top, x_axis="year",color="category2")

# Identify Anomalies in Daily Trend Data

In [24]:
from src.utils.gcp_utils import client
from src.ingest_data.cloud.gcp import get_bigquery_data

clickstream = get_bigquery_data(
    """
    SELECT date, sum(revenue) revenue 
    FROM `e-commerce-demo-v.summary.s_ga_organic`
    WHERE pagetype="product_page"
    GROUP BY 1
    ORDER BY 1 DESC
    """, 
    client
    )
clickstream.head()

,date,revenue
0,2017-08-01,NaN
1,2017-07-31,40.99
2,2017-07-30,38.79
3,2017-07-29,31.99
4,2017-07-28,11.99


In [25]:
from src.trend_data.anomaly import get_anomalous_records_std, get_anomaly_trend
clickstream_anomalies = get_anomalous_records_std(data = clickstream, date_key = "date", metric = "revenue")
get_anomaly_trend(clickstream_anomalies, metric = "revenue")

In [28]:
clickstream_aug_anomaly = get_bigquery_data(
    """
    SELECT entry_page, product_type, category_type, sum(revenue) revenue 
    FROM `e-commerce-demo-v.summary.s_ga_organic`
    WHERE date = "2016-10-20" and pagetype = "product_page"
    GROUP BY 1,2,3
    ORDER BY 4 DESC
    """, 
    client
    )
clickstream_aug_anomaly.head()

,entry_page,product_type,category_type,revenue
0,shop.googlemerchandisestore.com/google+redesig...,apparel,None,2000.76
1,shop.googlemerchandisestore.com/google+redesig...,accessories,None,NaN
2,shop.googlemerchandisestore.com/google+redesig...,office,None,NaN
3,shop.googlemerchandisestore.com/google+redesig...,other,None,NaN
4,shop.googlemerchandisestore.com/google+redesig...,apparel,None,NaN


In [27]:
clickstream_aug_anomaly['entry_page'][0]

'shop.googlemerchandisestore.com/google+redesign/apparel/men+s+google+performance+1/4+zip+hb.axd'

# Animate Trend Data

In [3]:
from src.utils.gcp_utils import client
from src.ingest_data.cloud.gcp import get_bigquery_data

clickstream_2016_pagetypes = get_bigquery_data(
    """
    SELECT date_month, pagetype, count(entry_page) as page_count, sum(pageviews) pageviews, sum(visits) visits, sum(revenue) revenue 
    FROM `e-commerce-demo-v.summary.s_ga_organic`
    WHERE date < "2017-12-31"
    GROUP BY 1,2
    ORDER BY 1 ASC
    """, 
    client
    )
clickstream_2016_pagetypes.head()

credential path: c:\Users\vanal\Desktop\repositories\analyst_trend_data\src\utils\..\..\config\gcp_service_account.json


,date_month,pagetype,page_count,pageviews,visits,revenue
0,2016-08-01,category_page,825,27284,5103,7863.25
1,2016-08-01,other,80,1484,228,1321.51
2,2016-08-01,policy_page,39,187,45,100.63
3,2016-08-01,basket_page,33,2714,171,4348.97
4,2016-08-01,None,0,0,7,NaN


In [4]:
from src.trend_data.trend import format_data_for_plotly
clickstream_2016_pagetypes_formated = format_data_for_plotly(clickstream_2016_pagetypes, date_key="date_month", kpi_list=["page_count","pageviews","visits","revenue"], dimension_list=["pagetype"])
clickstream_2016_pagetypes_formated

,date_month,pagetype,page_count,pageviews,visits,revenue
109,2017-08-01,brand_page,2,55,29,0.00
108,2017-08-01,sign_in_page,1,2,1,0.00
107,2017-08-01,category_page,11,110,19,0.00
106,2017-08-01,product_page,9,16,9,0.00
105,2017-08-01,home_page,4,243,46,39.28
...,...,...,...,...,...,...
1,2016-08-01,other,80,1484,228,1321.51
6,2016-08-01,brand_page,126,977,293,57.96
7,2016-08-01,home_page,122,104661,17467,43456.09
8,2016-08-01,sign_in_page,30,1349,106,1355.82


In [70]:
from src.trend_data.animate import get_trend_scatter_animation
get_trend_scatter_animation(clickstream_2016_pagetypes, x_axis="visits", y_axis="revenue", date_to_animate="date_month", animation_group="pagetype", color="pagetype")

In [5]:
from src.trend_data.animate import get_trend_bar_animation
get_trend_bar_animation(clickstream_2016_pagetypes, x_axis="pagetype", y_axis="revenue", date_to_animate="date_month", animation_group="pagetype", color="pagetype")